Text matching using characters
-----

Let's say you have the following strings:

1. "fuzzy rabbit"
2. "wuzzy rabbit"
3. "wuzzy wabbit"
4. "fuzzy wuzzy"

You want to cluster the first three strings together and the last string should be in a different cluster.

This is best done at the character level (an example of word-level semenatic clustering is [here](https://github.com/brianspiering/nlp-cookbook/blob/main/matching_text_with_embeddings.ipynb)).

Let's frame this problem within common used terminology. Hashing is mapping an object, a string in this case, to an integer. What you want are collisions (i.e., similar objects are mapped to the same integer bucket). The goal is to pick a hashing function that has higher number of collisions based on the number of shared letters in the string.

A scalable implementation of this idea is MinHash and locality-sensitive hashing (LSH). MinHash is rolling set hash of each individual character. LSH then clusters / buckets those hashes.

In [37]:
from dataclasses import dataclass, field

from datasketch import MinHash, MinHashLSH

@dataclass
class String:
    name:    str 
    text:    str
    minhash: object = field(init=False)
    
    def __post_init__(self):
        self.minhash = self.set_minhash()

    def set_minhash(self):
        "MinHash letter-by-letter"
        m = MinHash(num_perm=128)      
        for c in self.text:
            m.update(c.encode('utf8')) 
        return m

In [39]:
# Setup data
s0 =  String(name="s0", text="fuzzy rabbit")
s1 =  String(name="s1", text="wuzzy rabbit")
s2 =  String(name="s2", text="wuzzy wabbit")
s3 =  String(name="s3", text="fuzzy wuzzy")
strings = [s0, s1, s2, s3]

# Create LSH storage for scalable querying
lsh = MinHashLSH(threshold=0.8, num_perm=128)
for s in strings:
    lsh.insert(s.name, s.minhash)

# Test that the queries for the hashed values return the expected neighbors
assert set(lsh.query(s0.minhash)) == set(['s0', 's1', 's2'])
assert set(lsh.query(s3.minhash)) == set(['s3'])

This method is highly scalable. The hashing and LSH creation can be done in parrellel and offline. The querying is constant time O(1). The matching threshold needs to be custom tuned for the application.

<br>
<br>
<br>

------